In [43]:
import os
import pandas as pd
from constants import *
import hashlib

In [44]:
# Define o caminho para a pasta "dados"
data_folder = "dados"

# lista para armazenar os dataframes de cada arquivo CSV
dfs = []

# Percorre todos os arquivos .txt na pasta
for filename in os.listdir(data_folder):
    if filename.endswith('.txt'):
        # lê o arquivo como CSV com o pandas
        df = pd.read_csv(os.path.join(data_folder, filename), delimiter=';')

        # Verificar se o arquivo possui a coluna 'competênciaexc'
        if 'competênciaexc' not in df.columns:
            # Criar a coluna 'competênciaexc' com valor nulo
            df['competênciaexc'] = None

        # Verificar se o arquivo possui a coluna 'indicadordeexclusão'
        if 'indicadordeexclusão' not in df.columns:
            # Criar a coluna 'indicadordeexclusão' com valor 0
            df['indicadordeexclusão'] = 0
            
        # Criar a coluna data_mov
        df['competênciamov'] = df['competênciamov'].astype(str) + '01'
        df['data_mov'] = pd.to_datetime(df['competênciamov'], format='%Y%m%d')

        # criar colunas nu_ano e nu_mes
        df['nu_ano'] = df['data_mov'].dt.year
        df['nu_mes'] = df['data_mov'].dt.month

        # Alterar tipo de dados
        df['idade'] = df['idade'].fillna(0).astype(int)
        df['salário'] = df['salário'].str.replace(',', '.').fillna(0).astype(float)

        # Realizar de-para
        df['graudeinstrução'] = df['graudeinstrução'].astype(str).apply(lambda x: map_grau_instrucao.get(x, "Não Identificado"))
        df['sexo'] = df['sexo'].astype(str).apply(lambda x: map_genero.get(x, "Não Identificado"))
        df['raçacor'] = df['raçacor'].astype(str).apply(lambda x: map_etnia.get(x, "Não Identificado"))
        df['tipodedeficiência'] = df['tipodedeficiência'].astype(str).apply(lambda x: map_tp_deficiencia.get(x, "Não Identificado"))
        df['tipoempregador'] = df['tipoempregador'].astype(str).apply(lambda x: map_tp_empregador.get(x, "Não Identificado"))
        df['tipoestabelecimento'] = df['tipoestabelecimento'].astype(str).apply(lambda x: map_tp_estabelecimento[x])
        df['seção'] = df['seção'].astype(str).apply(lambda x: map_secao_economica.get(x, "Não Identificado"))
        df['subclasse'] = df['subclasse'].astype(str).apply(lambda x: map_subclasse_econ.get(x, "Não Identificado"))
        df['tamestabjan'] = df['tamestabjan'].astype(str).apply(lambda x: map_faixa_tam_estab.get(x, "Não Informado"))
        df['tipomovimentação'] = df['tipomovimentação'].astype(str).apply(lambda x: map_tp_movimentacao.get(x, "Não Identificado"))
        df['categoria'] = df['categoria'].astype(str).apply(lambda x: map_categoria.get(x, "Não Identificado"))
        df['cbo2002ocupação'] = df['cbo2002ocupação'].astype(str).apply(lambda x: map_cbo_ocupacao.get(x, "Não Identificado"))
        df['indtrabintermitente'] = df['indtrabintermitente'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indtrabparcial'] = df['indtrabparcial'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indicadoraprendiz'] = df['indicadoraprendiz'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indicadordeexclusão'] = df['indicadordeexclusão'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))
        df['indicadordeforadoprazo'] = df['indicadordeforadoprazo'].astype(str).apply(lambda x: map_sim_nao.get(x, "Não Identificado"))

        # Concatenar as colunas do DataFrame
        concatenacao_trabalhador = df['graudeinstrução'] + df['sexo'] + df['idade'].astype(str) + df['raçacor'] + df['tipodedeficiência']
        # Calcular o hash para a concatenação das colunas
        hashes_trabalhador = concatenacao_trabalhador.apply(lambda x: hashlib.md5(x.encode()).hexdigest())
        # Adicionar a coluna 'hash' ao DataFrame
        df['sk_trabalhador'] = hashes_trabalhador

        # Concatenar as colunas do DataFrame
        concatenacao_empregador = df['tipoempregador'] + df['tipoestabelecimento'] + df['seção'] + df['subclasse'] + df['tamestabjan']
        # Calcular o hash para a concatenação das colunas
        hashes_empregador = concatenacao_empregador.apply(lambda x: hashlib.md5(x.encode()).hexdigest())
        # Adicionar a coluna 'hash' ao DataFrame
        df['sk_empregador'] = hashes_empregador

        # Concatenar as colunas do DataFrame
        concatenacao_movimentacao = df['tipomovimentação'] + df['categoria'] + df['cbo2002ocupação'] + df['indtrabintermitente'] + df['indtrabparcial'] + df['indicadoraprendiz'] + df['indicadordeexclusão'] + df['indicadordeforadoprazo']
        # Calcular o hash para a concatenação das colunas
        hashes_movimentacao = concatenacao_movimentacao.apply(lambda x: hashlib.md5(x.encode()).hexdigest())
        # Adicionar a coluna 'hash' ao DataFrame
        df['sk_movimentacao'] = hashes_movimentacao

        # Renomar colunas
        df = df.rename(columns={
            'competênciamov': 'sk_periodo',
            'município': 'sk_localidade',
            'idade': 'nu_idade'
            })

        # Excluir coluna não usadas
        df = df.drop([
            'unidadesaláriocódigo',
            'valorsaláriofixo',
            'região',
            'uf',
            'origemdainformação',
            'competênciadec',
            'competênciaexc'
            ], axis=1)

        # Adiciona o dataframe à lista de dataframes
        dfs.append(df)

# concatena os dataframes em um único dataframe final
df_group = pd.concat(dfs, ignore_index=True)

In [ ]:
# exibe o dataframe final
pd.set_option('display.max_columns', None)
df_group.head()

,sk_periodo,sk_localidade,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,nu_idade,horascontratuais,raçacor,sexo,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,tamestabjan,indicadoraprendiz,indicadordeexclusão,indicadordeforadoprazo,data_mov,nu_ano,nu_mes,sk_trabalhador,sk_empregador,sk_movimentacao
0,20200101,352930,Saúde Humana e Serviços Sociais,Atividade Médica Ambulatorial Restrita a Consu...,1,"Recepcionista, em Geral",Empregado - Contrato a termo firmado nos termo...,Médio Completo,27,"44,00",Branca,Feminino,PJ,CNPJ,Admissão por reemprego,Não Deficiente,Não,Não,"1200,00",De 1 a 4,Não,Sim,Não,2020-01-01,2020,1,0e0f011ffb9f94a4f7b8cca9334f33db,51f4696066612a7075946909494147fb,150e0bcf561c91bb54ef8009af14ea88
1,20200101,330630,"Comércio, Reparação de Veículos Automotores e ...",Comércio Varejista de Material Elétrico,-1,Eletricista de Manutencao Eletroeletronica,"Empregado - Geral, inclusive o empregado públi...",Médio Completo,29,"42,50",Branca,Masculino,PJ,CNPJ,Desligamento por demissão sem justa causa,Não Deficiente,Não,Não,"1836,00",De 20 a 49,Não,Sim,Não,2020-01-01,2020,1,2f71fb6038bd71bef7f4bfaf35ef8ed3,c7a4fab4d064b8aaa98eaff213884aaa,d102ee27edb02420b98246d91ba8db3c
2,20200101,150140,"Comércio, Reparação de Veículos Automotores e ...",Comércio Atacadista de Cosméticos e Produtos d...,-1,Gerente Comercial,"Empregado - Geral, inclusive o empregado públi...",Médio Completo,39,"44,00",Parda,Feminino,PJ,CNPJ,Desligamento por demissão sem justa causa,Não Deficiente,Não,Não,"1998,00",De 1 a 4,Não,Sim,Não,2020-01-01,2020,1,b55b65f58a1a9976dbef4e548f75049c,1407d505637bbd69228f85ec7f96fb9d,c64f6bda192d6211e03948ed3a7afec1
3,20200101,330455,Saúde Humana e Serviços Sociais,Atividades de Apoio à Gestão de Saúde,-1,Agente Comunitario de Saude,Empregado - Contrato a termo firmado nos termo...,Médio Completo,48,"48,00",Preta,Feminino,PJ,CNPJ,Término contrato trabalho prazo determinado,Não Deficiente,Não,Não,"1400,00",1000 ou Mais,Não,Sim,Não,2020-01-01,2020,1,44d0fa25808d49819be4d6bc3d0c6134,91ae5f9a0cb8bbb56c9a23494f171720,ca5912a9d921b7e463b34f30a196d601
4,20200101,330455,Saúde Humana e Serviços Sociais,Atividades de Apoio à Gestão de Saúde,-1,Agente Comunitario de Saude,Empregado - Contrato a termo firmado nos termo...,Superior Incompleto,32,"48,00",Preta,Feminino,PJ,CNPJ,Término contrato trabalho prazo determinado,Não Deficiente,Não,Não,"1400,00",1000 ou Mais,Não,Sim,Não,2020-01-01,2020,1,6d295209cb349a12e7073037ecb32244,91ae5f9a0cb8bbb56c9a23494f171720,ca5912a9d921b7e463b34f30a196d601


In [ ]:
pd.set_option('display.max_rows', None)
df_group.dtypes

sk_periodo                        object
sk_localidade                      int64
seção                             object
subclasse                         object
saldomovimentação                  int64
cbo2002ocupação                   object
categoria                         object
graudeinstrução                   object
nu_idade                           int32
horascontratuais                  object
raçacor                           object
sexo                              object
tipoempregador                    object
tipoestabelecimento               object
tipomovimentação                  object
tipodedeficiência                 object
indtrabintermitente               object
indtrabparcial                    object
salário                           object
tamestabjan                       object
indicadoraprendiz                 object
indicadordeexclusão               object
indicadordeforadoprazo            object
data_mov                  datetime64[ns]
nu_ano          

In [ ]:
df_group.columns

Index(['sk_periodo', 'sk_localidade', 'seção', 'subclasse',
       'saldomovimentação', 'cbo2002ocupação', 'categoria', 'graudeinstrução',
       'nu_idade', 'horascontratuais', 'raçacor', 'sexo', 'tipoempregador',
       'tipoestabelecimento', 'tipomovimentação', 'tipodedeficiência',
       'indtrabintermitente', 'indtrabparcial', 'salário', 'tamestabjan',
       'indicadoraprendiz', 'indicadordeexclusão', 'indicadordeforadoprazo',
       'data_mov', 'nu_ano', 'nu_mes', 'sk_trabalhador', 'sk_empregador',
       'sk_movimentacao'],
      dtype='object')

In [ ]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948000 entries, 0 to 1947999
Data columns (total 29 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   sk_periodo              object        
 1   sk_localidade           int64         
 2   seção                   object        
 3   subclasse               object        
 4   saldomovimentação       int64         
 5   cbo2002ocupação         object        
 6   categoria               object        
 7   graudeinstrução         object        
 8   nu_idade                int32         
 9   horascontratuais        object        
 10  raçacor                 object        
 11  sexo                    object        
 12  tipoempregador          object        
 13  tipoestabelecimento     object        
 14  tipomovimentação        object        
 15  tipodedeficiência       object        
 16  indtrabintermitente     object        
 17  indtrabparcial          object        
 18  sa

In [ ]:
df_group.shape

(1948000, 29)

In [ ]:
pd.set_option('display.max_rows', None)
df_group.isna().sum()

sk_periodo                   0
sk_localidade                0
seção                        0
subclasse                    0
saldomovimentação            0
cbo2002ocupação              0
categoria                    0
graudeinstrução              0
nu_idade                     0
horascontratuais          5087
raçacor                      0
sexo                         0
tipoempregador               0
tipoestabelecimento          0
tipomovimentação             0
tipodedeficiência            0
indtrabintermitente          0
indtrabparcial               0
salário                      0
tamestabjan                  0
indicadoraprendiz            0
indicadordeexclusão          0
indicadordeforadoprazo       0
data_mov                     0
nu_ano                       0
nu_mes                       0
sk_trabalhador               0
sk_empregador                0
sk_movimentacao              0
dtype: int64

In [ ]:
# Criar df_periodo
df_periodo = df_group[['sk_periodo','data_mov','nu_ano','nu_mes']].drop_duplicates().reset_index(drop=True)
df_periodo.name = 'df_periodo'
df_periodo.head()

,sk_periodo,data_mov,nu_ano,nu_mes
0,20200101,2020-01-01,2020,1
1,20200201,2020-02-01,2020,2
2,20200301,2020-03-01,2020,3
3,20200401,2020-04-01,2020,4
4,20200501,2020-05-01,2020,5


In [ ]:
df_periodo.dtypes

sk_periodo            object
data_mov      datetime64[ns]
nu_ano                 int64
nu_mes                 int64
dtype: object

In [ ]:
# Define o caminho para a pasta "dados"
folder_path = "dados"


df_localidade = pd.read_csv(os.path.join(folder_path, 'MUNICIPIO.csv'), delimiter=';', encoding='ISO-8859-1')
# Renomear as colunas
df_localidade = df_localidade.rename(columns={'nm_regiao': 'is_regiao','nm_uf': 'is_uf','cd_mun_ibge_6': 'sk_localidade','nm_municipio': 'is_municipio'})
# Excluir colunas não usadas
df_localidade = df_localidade.drop(['cd_regiao','cd_uf','cd_mun_ibge_7'], axis=1)
# Corrigir a sequência
df_localidade = df_localidade.reindex(columns=['sk_localidade', 'is_regiao', 'is_uf','is_municipio'])

In [ ]:
df_localidade.head()

,sk_localidade,is_regiao,is_uf,is_municipio
0,110001,Norte,Rondônia,Alta Floresta D'Oeste
1,110037,Norte,Rondônia,Alto Alegre dos Parecis
2,110040,Norte,Rondônia,Alto Paraíso
3,110034,Norte,Rondônia,Alvorada D'Oeste
4,110002,Norte,Rondônia,Ariquemes
